In [ ]:
  import tensorflow as tf
  from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.utils import to_categorical
  from tensorflow.keras.callbacks import EarlyStopping
  from sklearn.model_selection import train_test_split

  # Load the dataset
  (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

  # Normalize the pixel values
  train_images = train_images / 255.0
  test_images = test_images / 255.0

  # Reshape the images to include channel dimension (grayscale)
  train_images = train_images.reshape((-1, 28, 28, 1))
  test_images = test_images.reshape((-1, 28, 28, 1))

  # Convert labels to one-hot encoding
  train_labels = to_categorical(train_labels)
  test_labels = to_categorical(test_labels)

  # Split the training data into training and validation sets
  train_images, val_images, train_labels, val_labels = train_test_split(
      train_images, train_labels, test_size=0.2, random_state=42
  )

  # Build the CNN model with dropout regularization
  model = Sequential([
      Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
      MaxPooling2D(2, 2),
      Conv2D(64, (3, 3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(128, activation='relu'),
      Dropout(0.5),  # Dropout layer for regularization
      Dense(10, activation='softmax')
  ])

  # Compile the model
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # Early stopping callback
  early_stopping = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.001, restore_best_weights=True)

  # Train the model with early stopping
  model.fit(
      train_images, train_labels,
      batch_size=32,
      epochs=20,
      validation_data=(val_images, val_labels),
      callbacks=[early_stopping]
  )

  # Evaluate the model on the test set
  test_loss, test_acc = model.evaluate(test_images, test_labels)
  print('Test accuracy:', test_acc)

4422102/4422102 [==============================] - 1s 0us/step
Epoch 1/20
1500/1500 [==============================] - 11s 4ms/step - loss: 0.6008 - accuracy: 0.7800 - val_loss: 0.3804 - val_accuracy: 0.8566
Epoch 2/20
1500/1500 [==============================] - 5s 4ms/step - loss: 0.4020 - accuracy: 0.8544 - val_loss: 0.3196 - val_accuracy: 0.8804
Epoch 3/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3478 - accuracy: 0.8752 - val_loss: 0.3088 - val_accuracy: 0.8841
Epoch 4/20
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3149 - accuracy: 0.8852 - val_loss: 0.2839 - val_accuracy: 0.8936
Epoch 5/20
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2886 - accuracy: 0.8957 - val_loss: 0.2696 - val_accuracy: 0.9011
Epoch 6/20
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2678 - accuracy: 0.9007 - val_loss: 0.2593 - val_accuracy: 0.9057
Epoch 7/20
1500/1500 [==============================] - 5s 3ms/step - lo

In [ ]:
print(f"The epoch with the lowest validation loss is: {early_stopping.stopped_epoch - early_stopping.patience + 1}")

The epoch with the lowest validation loss is: 10


In [ ]:
import ipywidgets as widgets
from IPython.display import display
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from io import BytesIO
import numpy as np
import tensorflow as tf

# Load your trained model here
# model = ...

# Assuming class_names is defined as before
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Function to load and preprocess an image from BytesIO
def load_and_preprocess_image_from_bytes(img_bytes):
    img = load_img(img_bytes, color_mode='grayscale', target_size=(28, 28))
    img = img_to_array(img)
    img = img.reshape(1, 28, 28, 1)  # Add batch dimension
    img = img / 255.0  # Normalize the image
    return img

# Function to handle file selection and make a prediction
def on_file_selected(change):
    if change['type'] == 'change' and change['name'] == 'value':
        uploaded_file = next(iter(change['new'].values()))  # Get the first uploaded file
        content = uploaded_file['content']
        img_bytes = BytesIO(content)

        img = load_and_preprocess_image_from_bytes(img_bytes)
        predictions = model.predict(img)
        predicted_class = np.argmax(predictions, axis=1)[0]

        print("Predicted class:", class_names[predicted_class])

# Create a file upload widget
file_picker = widgets.FileUpload(
    accept='image/png',  # Specify allowed file type
    multiple=False  # Allow single file selection only
)
file_picker.observe(on_file_selected, names='value')

# Display the file picker widget
display(file_picker)


FileUpload(value={}, accept='image/png', description='Upload')

1/1 [==============================] - 0s 226ms/step
Predicted class: Ankle boot
1/1 [==============================] - 0s 21ms/step
Predicted class: Dress
1/1 [==============================] - 0s 24ms/step
Predicted class: Dress
1/1 [==============================] - 0s 24ms/step
Predicted class: T-shirt/top
